# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

## API KEY 불러오기

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

## 대화 기록 함수 정의

In [4]:
model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [5]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 강민수야.")],
    config=config,
)

print(response.content)

안녕하세요, 강민수님! 만나서 반갑습니다. 어떻게 도와드릴까요?


## 두 번째 대화

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 강민수입니다! 어떻게 도와드릴까요?


## 세션 테스트

In [7]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 제가 당신의 이름을 알 수 있는 방법이 없습니다. 당신의 이름은 무엇인가요?'

In [8]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름이 강민수라고 이야기했습니다. 그 외에 어떤 내용을 얘기하고 싶으신가요? 혹시 다른 질문이 있으신가요?'

## 스트리밍 응답 출력

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")

강민수라는 이름은 주로 한국에서 사용되는 이름이기 때문에, 한국 사람이실 가능성이 높습니다. 한국 문화에 대해서 간단히 설명해 드리겠습니다.

한국 문화는 전통과 현대가 조화를 이루고 있습니다. 예를 들어, 전통적인 명절인 설날과 추석에는 가족들이 모여 음식을 나누고 조상을 기리는 제사를 지냅니다. 또한 식사 문화에서는 밥과 반찬을 함께 먹는 것이 일반적이며, 특히 김치가 한국 식사의 중요한 부분을 차지합니다.

한국의 대중문화 또한 세계적으로 인정받고 있습니다. K-pop, 드라마, 영화 등은 많은 팬을 보유하고 있으며, BTS와 같은 아티스트들은 글로벌한 인기를 끌고 있습니다.

한국의 전통 예술에도 관심을 가져볼 만합니다. 예를 들어, 한복(전통 의상), 한국 전통 음악(국악), 그리고 한국화와 같은 미술 장르들이 있습니다.

더 알고 싶은 점이나 특정한 질문이 있다면 말씀해 주세요!